<a href="https://colab.research.google.com/github/Zocha1/Introduction-To-Machine-Learning/blob/main/4_PRZYK%C5%81AD_Przetwarzanie_tekstu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Ładowanie zbioru danych

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


https://keras.io/api/datasets/imdb/

**'num_words=10000'** - użyjemy 10.000 najczęściej występujących słów

**'train_lables' 'test_labels'** - zmienne zawierające etykiety: 0 oznaczających recenzję negatywną i 1 oznaczających recenzję pozytywną



In [ ]:
train_data[0]

In [ ]:
train_labels[1]

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[1]])

print(decoded_review)

## 2. Przygotowanie danych
Do sieci neuronowej nie można wczytać list liczb całkowitych, trzeba je zamienić na listę **tensorów**. Istnieją dwa sposoby, które nam to umożliwią:
- Należy dopełnić listy, aby wszystkie były jednakowej długości i wtedy zamienić je na **tensor liczb całkowitych** o kształcie: (*próbki, indeksy_słów*). Wtedy pierwszą warstą sieci neuronowej musi być warstwa **Embedding**, która jest w stanie przetwarzać tensory liczb całkowitych.
- **One-hot encoding** - listy można także zamienić na wektory 0 i 1. Na przykład, jeśli twoja sekwencja to [3, 5], a masz słownik o rozmiarze 10 000 słów, to zapiszesz tę sekwencję jako wektor o długości 10 000, gdzie wszystkie elementy są zerami, oprócz indeksów 3 i 5, które są ustawione na 1. Wykorzystując tą metodę pierwszą warstwą sieci może być warstwa Dense, która jest w stanie obsługiwać wektory danych zmiennoprzecinkowych.

Korzystamy z drugiego rozwiązania:

Tworzymy tensor 2D, czyli macierz o wymiarach 25.000 x 10.000 wypełnioną 0 i 1. 25.000, bo tyle mamy recenzji i 10.000, bo tyle jest możliwych słów w recenzjach.
Przykładowo, jeśli recenzja składa się z trzech słów, które w naszym ograniczonym słowniku mają indeksy 5, 8 i 10, wówczas w wierszu odpowiadającym tej recenzji, w kolumnach 5, 8, 10 zostaną umieszczone jedynki, a reszta pozostanie zerami.


In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

Zanim przejdziemy do przetwarzania danych przez sieć neuronową, musimy jeszcze zamienić na wektory etykiety próbek.

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
y_train[0]

Po przetworzeniu danych, etykiety są wartościami skalarnymi - zera i jedynki, dane wejściowe wektorami.

## 3.Budowa sieci neuronowej

Głównym blokiem składowym sieci neuronowej jest **warstwa (layer)**. Można ją traktować jako filtr danych. Dane wychodzące z filtra mają bardziej przydatną formę od danych wchodzących.

Model uczenia głębokiego  jest jak sito przetwarzające dane składające się z coraz drobniejszych siatek-warstw.

Do rozwiązania naszego problemy najlepiej zastosować sieci prostego stosu w pełni połączonych warst - **Dense**:
- dwie pierwsze warsty z aktywacją - **relu**, zawierające 16 ukrytych jednostek (neuronów)
- ostatnia warstwa definijąca przewidywania sentymentu, czy recenzja będzie pozytywna, czy negatywna z funkcją aktywacji - **sigmoid**, która reprezentuje wartości w zakresie od 0 do 1, co pozwalana na określenie prawdopodobieństwa, zawierająca 1 neuron, ponieważ dla klasyfikacji binarnej wynik jest pojedynczą wartością prawdopodobieństwa przynależności do klasy pozytywnej

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Na tym etapie musimy określić jeszcze 3 rzeczy w celu przygotowania sieci do trenowania:
- **funkcja straty**: sposób pomiaru wydajności sieci podczas przetwarzania treningowego zbioru danych - dostrajanie parametrów sieci we właściwym kierunku
- **optymalizator**: mechanizm dostrajania sieci na podstawie danych zwracanych przez funkcję straty
- **metryki monitorowania podczas trenowania i testowania**: dokładność, czyli część obrazków, która została dobrze sklasyfikowana


- Nasz problem polega na klasyfikacji binarnej, w której sieć zwraca nam wartość prawdopodobieństwa, dlatego najlepszym wyborem funkcji straty będzie binarna entropia krzyżowa - **binary_crossentropy**.
- Dobrym wyborem do więkoszści problemów będzie optymalizator **rmsprop**.
- Metryką do zadań klasyfikacji binarnej jest dokładność - **accuracy**

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 4. Walidacja modelu


Walidacja modelu jest kluczowym krokiem w procesie budowania i optymalizacji modeli uczenia maszynowego, w tym sieci neuronowych. Jej głównym celem jest ocena, jak dobrze model generalizuje się na nowych, nieznanych danych, które nie były wykorzystywane podczas treningu.

W tym celu utworzymy zbiór danych, które nie były wykorzystane podczas trenowania modelu. Zrobimy to przez oddzielenie 10.000 próbek od treningowego zbioru danych.


In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

## 5. Trenowanie modelu

Teraz będziemy trenować model przez **20 epok**. To oznacza, że cały zbiór treningowy x_train i odpowiadające mu etykiety y_train zostaną przepuszczone przez model 20 razy, co pozwala na dokładniejsze dostosowanie wag modelu do przetwarzanych danych.

Z podziałem na **wsady po 512 próbek**. Oznacza to, że zamiast aktualizować wagi modelu po każdej próbce (co jest nieefektywne) lub po całym zestawie danych (co może wymagać dużo pamięci i być wolne), aktualizacje wag są przeprowadzane po każdych 512 próbkach. To ułatwia efektywniejsze i szybsze treningi.

Jednocześnie będziemy monitorować funkcje straty i dokładność modelu przy przetwarzaniu 10.000 próbek, które przed chwilą wyodrębniliśmy. W tym celu musimy przekazać zbiór walidacyjny (kontrolny) jako argument validation_data.
Pod koniec trwania każdej epoki algorytm zatrzymuje się na chwilę, ponieważ model oblicza stratę i dokładność, korzystając z 10.000 próbek walidacyjnego zbioru danych.



In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Wywołanie metody **model.fit()** zwraca obiekte **History**, zawierający element **history** - słownik danych dotyczących przebiegu trenowania.

In [ ]:
history_dict = history.history
history_dict.keys()

Ocena wydajności już wytrenowanego modelu na zestawie danych testowych, pozwala na sprawdzenie, jak dobrze model generalizuje się na nowych, niewidzianych wcześniej danych.

In [ ]:
results = model.evaluate(x_test, y_test)

## 6. Interpretacja wyników przewidywań

In [ ]:
import matplotlib.pyplot as plt

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label='Strata trenowania')
plt.plot(epochs, val_loss_values, 'b', label='Strata walidacji')
plt.title('Strata trenowania i walidacji')
plt.xlabel('Epoki')
plt.ylabel('Strata')
plt.legend()

plt.show()

In [ ]:
plt.clf()

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Dokładność trenowania')
plt.plot(epochs, val_acc_values, 'b', label='Dokładność walidacji')
plt.title('Dokładność trenowania i walidacji')
plt.xlabel('Epoki')
plt.ylabel('Dokładność')
plt.legend()

plt.show()